# 응용통계학 과제
> 다중공선성

- toc:true
- branch: master
- badges: true
- comments: false
- author: 최서연
- categories: [Applied Statistics, 다중공선성]

# 응용통계학 과제 20220512
- 202150754 최서연
> 다중공선성 


다중공선성이 존재하는 상황을 가정하고 

 

다중공선성을 어느 정도 제거한 모형 (M1)과 다중공선성이 내재되어 있는 모형 (M2) 을 고려하여

 

두 모형의 예측력을 모의실험을 통해 비교하여라,

 

단, 실험은 여러 번 반복하여 평균적인 결과를  report하되 설명변수의 개수는 3개 이상으로 설정하여라.

 

이미 존재하는 문서들을 참고하거나 재현해도 무방함.

 

(첨부된 문서 참고)

In [131]:
Nrep = 200

n = 1000

hatb1 <- hatb2 <- c()
te1 <- te2 <- c()

for (k in 1:Nrep)
{
x1 = runif(n,-3,3)
x2 = x1 + rnorm(n,0,0.01)
#X = model.matrix(~ x1 + x2)
#solve(t(X)%*%X)
y = 2 + x1 + 2*x2 + rnorm(n)

ind = sample(1:n,500)
tx1 = x1[ind]
tx2 = x2[ind]
test_x1 = x1[-ind]
test_x2 = x2[-ind]
ty  = y[ind]
test_y = y[-ind]

fit1 = lm(ty~tx1+tx2)
fit2 = lm(ty~tx1)

hatb1[k] = fit1$coefficients[2]
hatb2[k] = fit2$coefficients[2]

te1[k] = mean((test_y - predict(fit1,newdata=data.frame(test_x1,test_x2)))^2)
te2[k] = mean((test_y - predict(fit2,newdata=data.frame(test_x1)))^2)
#summary(lm(y~x1+x2))
#print(k)
}

c(mean(te1),mean(te2))
c(mean((hatb1-1)^2),mean((hatb2-2)^2))

[1] 54.98598 54.98444

[1] 21.033269  1.000548

---

In [165]:
library(regclass)

---

## 시도 1

In [223]:
x1 = runif(500)
x2 = x1 + rnorm(500,0,0.01)
x3 = x1 + rnorm(500,0,0.1)
y1= 3 + x1 + 1.5*x2 + 3.5*x3 + rnorm(500)

In [224]:
cor(x1,x2)
cor(x1,x3)

[1] 0.999398

[1] 0.9472068

높은 상관계수 확인

In [225]:
M1 = lm(y1~x1)

In [227]:
M2 = lm(y1~x1+x2+x3)

In [229]:
VIF(M2)

x1         x2         x3 
843.200649 831.081927   9.731552

다중공선성 존재를 가정한 모형의 VIF 10 넘는 모습이었다.

In [230]:
M1$coefficients

(Intercept)          x1 
   3.139089    5.724312

In [232]:
M2$coefficients

(Intercept)          x1          x2          x3 
   3.155258   -1.186232    3.552497    3.383659

In [233]:
mean((y1-predict(M1,data.frame(x1)))^2)

[1] 1.111427

In [235]:
mean((y1-predict(M2,data.frame(x1,x2,x3)))^2)

[1] 1.003206

다중공선성을 제거한 모형이 다중공선성이 있는 모형보다 제곱평균오차가 컸다.

---

## 시도 2

In [193]:
Nrep = 200

n = 1000

hatb1 <- hatb2 <- c()
te1 <- te2 <- c()

for (k in 1:Nrep)
{
x1 = runif(n,-3,3)
x2 = x1 + rnorm(n,0,0.01)
x3 = x1^2 + rnorm(n,0,0.01)
y = 2 + x1 + 2*x2 + 3*x3 + rnorm(n)

ind = sample(1:n,500)
tx1 = x1[ind]
tx2 = x2[ind]
tx3 = x3[ind]
test_x1 = x1[-ind]
test_x2 = x2[-ind]
test_x3 = x3[-ind]
ty  = y[ind]
test_y = y[-ind]

M2 = lm(ty~tx1+tx2+tx3)
M1 = lm(ty~tx1)

hatb1[k] = M2$coefficients[2]
hatb2[k] = M1$coefficients[2]

te1[k] = mean((test_y - predict(M1,newdata=data.frame(test_x1,test_x2,test_x3)))^2)
te2[k] = mean((test_y - predict(M1,newdata=data.frame(test_x1)))^2)
}

c(mean(te1),mean(te2))
c(mean((hatb1-1)^2),mean((hatb2-2)^2))

[1] 119.4893 119.4893

[1] 21.769712  1.094602

다중공선성이 있는 모형과 다중공선성이 없는 모형의 MSE가 같았다.